<a href="https://colab.research.google.com/github/marcosnevary/computational-mathematics/blob/main/colab/01_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Avaliação Alternativa 1**

In [ ]:
import sys

sys.path.append('../')

In [ ]:
import numpy as np
import pandas as pd
from src.utils.binary_multiply import binary_multiply
from src.utils.binary_to_decimal import binary_to_decimal

## Problema 1

<div style="text-align: center;">
    <img src="https://github.com/marcosnevary/computational-mathematics/blob/main/src/images/bisection_method_graph.png?raw=True" width="600">
</div>

**Teorema:** Seja $f(x)$ uma função contínua num intervalo $[a, b]$. Se $f(a)f(b) < 0$, então existe pelo menos um ponto $x = \alpha$, onde $\alpha \in (a, b)$, tal que $f(\alpha) = 0$.
1. $[a, b]$ é o intervalo inicial, $\epsilon$ é o erro objetivo, $k = 0$.
2. Faça $x =  \frac{a + b}{2}$
3. Se $f(a)f(x) < 0$, faça $b = x$. Caso contrário, faça $a = x$.
4. Se $b - a < \epsilon$, então escolha um $\overline x$ qualquer em $[a, b]$ e pare. Caso contrário $k = k + 1$ e volte ao passo 2.

In [ ]:
def split_intervals(derivative, a, b):
    sample_points = np.linspace(a, b)
    derivative_values = [derivative(x) for x in sample_points]

    if all(d >= 0 for d in derivative_values) or all(d <= 0 for d in derivative_values):
        return [(a, b)]

    mid = (a + b) / 2

    left_intervals = split_intervals(derivative, a, mid)
    right_intervals = split_intervals(derivative, mid, b)

    return left_intervals + right_intervals

def bisection_method(function, derivative, step_size):
    intervals = []
    x_grid = np.arange(-101, 101, step_size)

    for x in x_grid:
        if (function(x) <= 0 and function(x + step_size) >= 0) or (function(x) >= 0 and function(x + step_size) <= 0):
            intervals.extend(split_intervals(derivative, x, x + step_size))

    roots = []

    for interval in intervals:
        print(f'Intervalo: {(float(interval[0]), float(interval[1]))}')
        a = interval[0]
        b = interval[1]
        epsilon = 0.001

        log = {'Iteração': [], 'x': [], 'f(x)': [], 'b - a': []}

        iteration = 1
        while b - a > epsilon:
            x = (a + b) / 2

            if function(a) * function(x) < 0:
                b = x
            else:
                a = x

            log['Iteração'].append(iteration)
            log['x'].append(x)
            log['f(x)'].append(function(x))
            log['b - a'].append(b - a)

            iteration += 1

        df = pd.DataFrame(log)
        display(df.set_index('Iteração'))

        roots.append(float((a + b) / 2))
        print()

    return roots

In [ ]:
def function(x):
    return x ** 3 - 9 * x ** 2 + 3

def derivative(x):
    return 3 * x ** 2 - 18 * x

roots = bisection_method(function, derivative, 1)
print(f'Raízes encontradas: {roots}')

Intervalo: (-1.0, 0.0)


,x,f(x),b - a
Iteração,,,
1,-0.500000,0.625000,0.500000
2,-0.750000,-2.484375,0.250000
3,-0.625000,-0.759766,0.125000
4,-0.562500,-0.025635,0.062500
5,-0.531250,0.310028,0.031250
6,-0.546875,0.144794,0.015625
7,-0.554688,0.060231,0.007812
8,-0.558594,0.017461,0.003906
9,-0.560547,-0.004046,0.001953



Intervalo: (0.0, 1.0)


,x,f(x),b - a
Iteração,,,
1,0.500000,0.875000,0.500000
2,0.750000,-1.640625,0.250000
3,0.625000,-0.271484,0.125000
4,0.562500,0.330322,0.062500
5,0.593750,0.036469,0.031250
6,0.609375,-0.115757,0.015625
7,0.601562,-0.039205,0.007812
8,0.597656,-0.001258,0.003906
9,0.595703,0.017633,0.001953



Intervalo: (8.0, 9.0)


,x,f(x),b - a
Iteração,,,
1,8.500000,-33.125000,0.500000
2,8.750000,-16.140625,0.250000
3,8.875000,-6.845703,0.125000
4,8.937500,-1.992432,0.062500
5,8.968750,0.486298,0.031250
6,8.953125,-0.757427,0.015625
7,8.960938,-0.136656,0.007812
8,8.964844,0.174548,0.003906
9,8.962891,0.018877,0.001953



Raízes encontradas: [-0.56005859375, 0.59716796875, 8.96240234375]


## Problema 2

A sequência binária $00111110010100000000000000000000$ (32 bits) pode ser transformada em um número de ponto flutuante no padrão IEEE 754 de precisão simples através do seguinte processo:

- O primeiro bit ($0$) é o bit de sinal. Se ele for $0$, o número é positivo (sinal é igual a $1$), se for $1$ o número é negativo (sinal é igual a $-1$).

- Do segundo ao nono bit ($01111100$) é o número binário que representa o expoente armazenado. O expoente real é calculado pela diferença entre o expoente armazenado e o bias, onde o bias é calculado através da fórmula $2^{k-1} - 1$, onde $k$ é a quantidade de bits reservada para o expoente ($k = 8$ para precisão simples), logo o bias é igual a $127$.

- Do décimo bit em diante ($10100000000000000000000$) estão os bits da mantissa (parte fracionária do número binário).

A parte inteira da mantissa é igual a $1$, logo a conversão do número binário fracionário começa com $1$ e, em seguida, soma-se cada bit da mantissa multiplicado por $2^{-i}$, onde $i$ é a posição desse bit na mantissa. A conversão final é dada pelo sinal multiplicado pela mantissa multiplicado por $2$ elevado ao expoente real.


In [ ]:
def ieee754(binary_sequence):
    sign_bit = binary_sequence[0]
    exponent_bits = binary_sequence[1:9]
    mantissa_bits = binary_sequence[9:]

    if sign_bit == '1':
        sign = -1
    else:
        sign = 1

    exponent = binary_to_decimal(exponent_bits) - 127

    mantissa = 1

    for index, bit in enumerate(mantissa_bits, start=1):
        mantissa += int(bit) * (2 ** -index)

    return sign * mantissa * (2 ** exponent)

In [ ]:
binary_sequences = ['00111110010100000000000000000000', '01000001011100000000000000000000']

for binary_sequence in binary_sequences:
    decimal = ieee754(binary_sequence)
    print(f'({binary_sequence})_2 = ({decimal})_10')

(00111110010100000000000000000000)_2 = (0.203125)_10
(01000001011100000000000000000000)_2 = (15.0)_10


## Problema 3

Considere um número binário entre 0 e 1 $n = (0.a_0a_1\dots a_{j - 1}a_j\dots)_2$, a sua representação decimal $(0.b_0b_1\dots b_{j - 1}b_j\dots)_{10}$ é obtida através do processo:
1. $k = 0$, $n_k = n$
2. Calcule $w_k = (1010)_2 \cdot n_k$.
3. Seja $z_k$ a parte inteira de $w_k$.
4. $b_k$ é a conversão de $z_k$ para a base 10.
5. Faça $n_{k +1} = w_k - z_k$.
6. Se $n_{k + 1} = 0$, pare. Caso contrário, faça $k = k + 1$ e repita a partir do passo 2.

Por exemplo, o número binário $(0.000111)_2$ é igual ao número decimal $(0.109375)_{10}$. Podemos visualizar essa transformação através do código abaixo.

In [ ]:
def binary_fraction_to_decimal(binary_fraction):
    remainder = binary_fraction
    decimal = 0
    position = -1

    while remainder != '0':
        product = binary_multiply('1010', remainder)

        if '.' in product:
            integer, remainder = product.split('.')
            digit = binary_to_decimal(integer)
            decimal += digit * (10 ** position)

            if '.' not in remainder:
                remainder = '0.' + remainder
        else:
            digit = binary_to_decimal(product)
            decimal += digit * (10 ** position)
            remainder = '0'

        position -= 1

    return decimal

In [ ]:
binary = '0.000111'
decimal = binary_fraction_to_decimal(binary)
print(f'({binary})_2 = ({decimal})_10')

(0.000111)_2 = (0.10937500000000001)_10
